## PetFinder.my - Pawpularity Contest
---

In [ ]:
!pip install '../input/timm-whl/timm-0.4.12-py3-none-any.whl'

In [ ]:
import os

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')

In [ ]:
!cp '../input/timm-whl/efficientnet_b0_ra-3dd342df.pth' '/root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth'

In [ ]:
import numpy as np
import pandas as pd

import gc

import cv2

# plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from tqdm.notebook import tqdm_notebook

from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from sklearn.model_selection import StratifiedKFold

init_notebook_mode(connected=True)

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.optim as optim

import timm

In [ ]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')

In [ ]:
train_df.shape

In [ ]:
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
fig = px.histogram(train_df, x='Pawpularity')
fig.update_layout(autosize=False, width=700)
fig.show()

In [ ]:
metadata = ["Subject Focus", "Eyes", "Face", "Near", "Action", "Accessory", \
            "Group", "Collage", "Human", "Occlusion", "Info", "Blur"]

fig = make_subplots(rows=2, cols=6)

for i, name in enumerate(metadata):
    row, col = i // 6, i % 6
    trace1 = go.Box(x=train_df['Near'], y=train_df['Pawpularity'], name=name)
    fig.append_trace(trace1, row = row + 1, col = col + 1)
    
fig.show()

In [ ]:
# image related utility functions

def image_read(path):
    img = cv2.imread(path)
    if img is None:
        raise Exception("Unable to read an image")
    return img

def read_image_rgb(path):
    img = image_read(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def image_h_w(path):
    img = image_read(path)
    shape = img.shape
    return shape[0], shape[1]

In [ ]:
train_df['path'] = train_df.apply(lambda x : '../input/petfinder-pawpularity-score/train/' + x['Id'] + ".jpg", axis=1)
train_df['image_h_w'] = train_df.apply(lambda x : image_h_w(x.path), axis=1)
train_df['height'] = train_df.apply(lambda x : x['image_h_w'][0], axis=1)
train_df['width'] = train_df.apply(lambda x : x['image_h_w'][1], axis=1)

In [ ]:
fig = px.scatter(x=train_df['width'], y=train_df['height'])
fig.update_layout(autosize=False, width=700)
fig.show()

In [ ]:
# Plot first 50 images
fig, ax = plt.subplots(5,10,figsize=(16, 10))
for i, (path, score) in enumerate(train_df[['path', 'Pawpularity']][:50].values.tolist()):
    row, col = i // 10, i % 10
    axis = ax[row][col]
    axis.imshow(image_read(path))
    axis.set_xticks([])
    axis.set_yticks([])
    axis.set_xlabel(score)
plt.show()

In [ ]:
class PetDataset(Dataset):
    
    def __init__(self, paths, labels=None, transforms=None):
        self.paths = paths
        self.labels = labels
        self.transforms = transforms
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx : int):
        
        img = read_image_rgb(self.paths[idx])
        
        if self.transforms:
            img = self.transforms(img)
    
        # if we use for model inference  
        if self.labels is None:
            return img
        
        return img, self.labels[idx]
    

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def sigmoid_fn(x):
    return 1/(1 + np.exp(-x))


def rmse_fn(predictions, targets):
    return np.sqrt(
        np.mean((predictions-targets)**2)
    )


class PetNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True)
        self.model.classifier = nn.Linear(in_features=1280, out_features=1)

    def forward(self, x):
        return self.model(x)


class PetTrainer:

    def __init__(self, net) -> None:
        self.net = net.to(device)
        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer_fn = optim.AdamW(params=self.net.parameters())

    def train(self, data_loader):

        # model set to train mode
        self.net.train()

        total_loss = 0

        for img_batch, label_batch in tqdm_notebook(data_loader):

            img_batch, label_batch = \
                img_batch.to(device), label_batch.to(device)

            # reset params
            self.optimizer_fn.zero_grad()

            # forward
            output = self.net(img_batch)

            # compute loss and take backward
            loss_op = self.loss_fn(output, label_batch)
            loss_op.backward()

            # take optimizer step
            self.optimizer_fn.step()

            total_loss += loss_op.item()

        print(f"BCELoss: {total_loss / len(data_loader)}")

    def test(self, data_loader):

        self.net.eval()

        predict_ls, true_ls = [], []

        with torch.no_grad():
            for img_batch, label_batch in tqdm_notebook(data_loader):
                img_batch = img_batch.to(device)
                prediction = self.net(img_batch).cpu()
                predict_ls.extend(prediction.numpy())
                true_ls.extend(label_batch.numpy())

        predict_ls = np.array(predict_ls).ravel()
        predict_ls = sigmoid_fn(predict_ls) * 100

        true_ls = np.array(true_ls).ravel() * 100

        rmse = rmse_fn(predict_ls, true_ls)

        print(predict_ls[:50])
        print(true_ls[:50])
        print("RMSE:", rmse)

        return rmse

    def infer(self, data_loader):
        
        self.net.eval()
        
        predict_ls = []
        with torch.no_grad():
            for img_batch in tqdm_notebook(data_loader):
                img_batch = img_batch.to(device)
                prediction = self.net(img_batch).cpu()
                predict_ls.extend(prediction.numpy())
                
        predict_ls = np.array(predict_ls).ravel()
        predict_ls = sigmoid_fn(predict_ls) * 100
        
        return predict_ls


In [ ]:
# Note:
# ToTensor() -> change channel order from (H x W x C) to (C x H x W), and scale from [0,255] -> [0,1]
# Normalize() -> mean, std, will convert to z-score, y = (x-mean)/std

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
#         transforms.RandomResizedCrop(224),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

train_df['stratify_label'] = pd.qcut(train_df['Pawpularity'], q = 30, labels = range(30))

X_data = np.array(train_df.path)
y_data = np.array(train_df.Pawpularity)
y_data = y_data / 100.0
y_data = np.expand_dims(y_data, axis=1)
y_skflabel = np.array(train_df.stratify_label)


debug = False
epochs = 20
train_batch_size = 128
test_batch_size = 32
n_splits = 5

if debug: epochs = 5

In [ ]:
!mkdir models

In [ ]:
skf = StratifiedKFold(n_splits=n_splits, random_state=2022, shuffle=True)

checkpoints = []

for fold_i, (train_index, test_index) in enumerate(skf.split(X_data, y_skflabel)):
    
    print("FOLD:", fold_i + 1)

    X_train, y_train = X_data[train_index], y_data[train_index]
    X_test, y_test = X_data[test_index], y_data[test_index]
    
    train_dataloader = DataLoader(PetDataset(X_train, y_train, data_transforms['train']), batch_size=train_batch_size)
    test_dataloader = DataLoader(PetDataset(X_test, y_test, data_transforms['test']), batch_size=test_batch_size)

    print(len(train_dataloader.dataset), len(test_dataloader.dataset))
    
    # clean cache and model
    try: del net
    except Exception: pass
    gc.collect()
    torch.cuda.empty_cache()

    net = PetNet()

    trainer = PetTrainer(net)
    for i in range(epochs):
        trainer.train(train_dataloader)
        
    score = trainer.test(test_dataloader)
    
    out_path = f"models/model{fold_i}.pt"
    print("Model saving at:", out_path)
    torch.save(net.state_dict(), out_path)
    
    checkpoints.append((score, out_path))
    
    if debug:
        break

In [ ]:
checkpoints

In [ ]:
best_scored = sorted(checkpoints)[0]
best_scored, best_scored[1]

In [ ]:
net = PetNet()
path = best_scored[1]
print("Model Loading:", path)
net.load_state_dict(torch.load(path))

trainer = PetTrainer(net)

In [ ]:
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df['path'] = test_df.apply(lambda x : '../input/petfinder-pawpularity-score/test/' + x['Id'] + ".jpg", axis=1)
X_pred_data = np.array(test_df.path)
pred_dataloader = DataLoader(PetDataset(X_pred_data, None, data_transforms['test']), batch_size=test_batch_size)

In [ ]:
pred = trainer.infer(pred_dataloader)

In [ ]:
pred[:10]

In [ ]:
submit_df = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
submit_df.Pawpularity = pred
submit_df.to_csv("submission.csv", index=False)